# Problema del commesso viaggatore (TSP)

## Definizione
### Dato un grafo $G(V,A)$ a cui è associato un costo $d_{ij}$ ad ogni arco $(i,j) \in A$. Il problema del commesso viaggiatore consiste nel cercare il circuito Hamiltoniano di costo minimo.
### - Un ciclo Hamiltoniano è un ciclo che attraversa tutti i nodi del grafo una ed una sola volta.
### - Il costo di un ciclo Hamiltoniano è dato dalla somma dei costi degli archi che lo compongono.

## Definizione

## Variabili di decisione

### <span style="color:purple">$x_{ij} \quad (i,j) \in A $ </span>  - variabile binaria uguale a $1$ se l'arco $(i,j)$ appartiene al circuito hamiltoniano, 0 altrimenti.


## Funzione obiettivo
### Minimizza il costo totale del circuito hamiltoniano

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in A} d_{ij} \cdot x_{ij}
\end{equation}

## Vincoli di assegnamento
### In una soluzione ammissibile (circuito hamiltoniano) ogni nodo deve avere esattamente un arco entrante ed esattamente un arco uscente.

\begin{equation}
\sum_{i \in V, \ i \neq j} x_{ij} = 1 \quad \quad j \in V 
\end{equation}

\begin{equation}
\sum_{i \in V, \ i \neq j} x_{ji} = 1 \quad \quad j \in V 
\end{equation}

## Vincoli di assenza di sottogiri 

\begin{eqnarray}
& \sum_{i,j \in S} x_{ij} \leq |S| -1 \quad S \subset V
\end{eqnarray}

## Istanza

### Legge il file in input e crea il dizionario *Nodes* con chiave *nome del nodo* e valore *coordinate del nodo*

In [1]:
Nodes = {}

file = open("att48.tsp", "r")

line = file.readline()

while not line.startswith("EOF"):
    
    if line.startswith("DIMENSION"):
        NumNodes = int(line.split()[1])
    elif line.startswith("NODE"):
        for i in range(NumNodes) :
            line = file.readline().split()
            Nodes[int(line[0])] = (float(line[1]), float(line[2]))
    
    line = file.readline()

### Crea il dizionario Dist con chiave *(i,j)* e valore la distanza euclidea

In [2]:
import math

Dist = {}

for i in Nodes :
    for j in Nodes :
        if i != j :
            difx = Nodes[i][0] - Nodes[j][0]
            dify = Nodes[i][1] - Nodes[j][1]
            Dist[(i,j)] = math.sqrt(difx**2 + dify**2)

## Importa la libreria GRB inizializza il problema e definisce le variabili

In [3]:
import gurobipy as gp
from gurobipy import GRB

mod = gp.Model("TSP")

Xvars = mod.addVars(Dist.keys(), obj = Dist, vtype = GRB.BINARY, name = "x")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-26


## Vincoli di assegnamento

In [4]:
outstar = mod.addConstrs(gp.quicksum(Xvars[i,j] for j in Nodes if i != j) == 1 for i in Nodes)

instar = mod.addConstrs(Xvars.sum('*',i) == 1 for i in Nodes)

## Procedura che individua i sottogiri

### input
- SOL -> lista degli archi nella soluzione corrente

### output
- feasible -> = True se la soluzione è ammissibile
- SubTour -> lista dei nodi del sottogiro individuato

In [5]:
def LookForSubTours(SOL, FirstNode):

    feasible = True

    UnVisited = list(Nodes.keys())
    Visited = []
    NextNode = FirstNode
    
    while NextNode not in Visited :
        
        CurrentNode = NextNode
        UnVisited.remove(CurrentNode)
        Visited.append(CurrentNode)
        
        for (i,j) in SOL :
            if i == CurrentNode :
                NextNode = j
                break
        
    if len(UnVisited) > 0 :
        feasible = False
        
    return feasible, Visited


def LookForMinSubTour(SOL):
    
    UnVisited = list(Nodes.keys())
    MinTour = list(Nodes.keys())
    
    while len(UnVisited) > 0 :
        
        FirstNode = UnVisited[0]
        feasible, SubTour = LookForSubTours(SOL, FirstNode)
        
        if len(SubTour) <= len(MinTour):
            MinTour = SubTour
        
        for i in SubTour :
            UnVisited.remove(i)
        
    return feasible, MinTour

In [6]:
def GeneraLazyConstr(mod, where) :
    
    if where == GRB.Callback.MIPSOL :
        
        SOL = []
        Xvals = mod.cbGetSolution(mod._Xvars)
        for (i,j) in Xvals:
            if Xvals[i,j] > 0.5 :
                SOL.append((i,j))
        
        feasible, Tour = LookForMinSubTour(SOL)
        
        if not feasible :
            mod.cbLazy(gp.quicksum(Xvars[i,j] for i in Tour for j in Tour if i != j) <= len(Tour) -1)

## Risolve il modello

In [7]:
mod.setParam("OutputFlag", 1)
mod._Xvars = Xvars
mod.setParam
mod.Params.LazyConstraints = 1
mod.optimize(GeneraLazyConstr)


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 96 rows, 2256 columns and 4512 nonzeros
Model fingerprint: 0xcd1ba8d4
Variable types: 0 continuous, 2256 integer (2256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 96 rows, 2256 columns, 4512 nonzeros
Variable types: 0 continuous, 2256 integer (2256 binary)

Root relaxation: objective 2.788971e+04, 87 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 27889.7096    0    -          - 27889.

## Preleva la soluzione e crea una lista *sortedPos* che contiene le posizioni dei nodi nel tour 

In [9]:
print("Valore Soluzione =", mod.ObjVal)

SOL = []
print("Soluzione ottima:")
for (i,j) in Xvars:
    if Xvars[i,j].X > 0.5 :
        SOL.append((i,j))
print(SOL)

Valore Soluzione = 33523.708507435585
Soluzione ottima:
[(1, 8), (2, 29), (3, 23), (4, 26), (5, 42), (6, 37), (7, 28), (8, 38), (9, 1), (10, 45), (11, 12), (12, 15), (13, 11), (14, 25), (15, 40), (16, 22), (17, 43), (18, 7), (19, 27), (20, 47), (21, 32), (22, 3), (23, 14), (24, 10), (25, 13), (26, 2), (27, 17), (28, 6), (29, 34), (30, 36), (31, 44), (32, 39), (33, 20), (34, 41), (35, 4), (36, 46), (37, 19), (38, 31), (39, 48), (40, 9), (41, 16), (42, 24), (43, 30), (44, 18), (45, 35), (46, 33), (47, 21), (48, 5)]


### Didegna la soluzione

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(20,20))
for i in Nodes:
    plt.scatter(Nodes[i][0],Nodes[i][1])
    plt.text(Nodes[i][0],Nodes[i][1], str(i),fontsize=18)
            

for i in range(NumNodes -1):
      plt.plot([Nodes[sortedPos[i]][0],Nodes[sortedPos[i+1]][0]],
               [Nodes[sortedPos[i]][1],Nodes[sortedPos[i+1]][1]])  
            
plt.plot([Nodes[sortedPos[len(sortedPos)-1]][0],Nodes[sortedPos[0]][0]],
         [Nodes[sortedPos[len(sortedPos)-1]][1],Nodes[sortedPos[0]][1]])
        
plt.xlabel('X');
plt.ylabel('Y');